In [121]:
import pandas as pd
import numpy as np
import itertools as it 
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel as C
from sklearn.pipeline import Pipeline

from ast import literal_eval

# Guide and Ideas for Capstone Project

We begin this guide by downloading the data:

In [122]:
#PARAMETERS 
# This section defines the variables to run the rest of the document 
file = 8  # Identifies the function number to execute (1,2,3...8) 
resultsFile = "data/results10.csv"   # The latest results file provided by Carlton on a weekly basis 
#kernel_name = "lineal"   # Options are default, linear and polynomial

In [123]:
inputColumnName = "f" + str(file)
outputColumnName = "f" + str(file) + "_output"
df = pd.read_csv(resultsFile)
input = df[inputColumnName]
output = df[outputColumnName]
print("Input=", input)
print("Output=", output)
df.head(25)


Input= 0    [0.067737 0.079147 0.027514 0.046544 0.461641 ...
1    [0.081284 0.094976 0.033017 0.055853 0.369313 ...
2    [0.081284 0.094976 0.033017 0.055853 0.553969 ...
3    [0.097541 0.113971 0.03962  0.067024 0.443175 ...
4    [0.117049 0.136765 0.047544 0.080429 0.53181  ...
5    [0.140459 0.109412 0.057053 0.096515 0.425448 ...
6    [0.093639 0.164118 0.057053 0.096515 0.486226 ...
7    [0.074911 0.131294 0.068464 0.115818 0.555687 ...
8    [0.059929 0.105035 0.054771 0.092654 0.666824 ...
9    [0.071915 0.084028 0.065725 0.074123 0.647772 ...
Name: f8, dtype: object
Output= 0    9.799602
1    9.696555
2    9.817107
3    9.849910
4    9.859542
5    9.833724
6    9.865061
7    9.906853
8    9.947503
9    9.953320
Name: f8_output, dtype: float64


,Unnamed: 0,timestamp,student_id,f1,f2,f3,f4,f5,f6,f7,f8,f1_output,f2_output,f3_output,f4_output,f5_output,f6_output,f7_output,f8_output
0,24,2024-05-05 12:53:51,534,[0.877228 0.8796 ],[0.562109 0.999999],[0.463899 0.489275 0.408212],[0.462212 0.386621 0.366643 0.298809],[0.269027 0.999999 0.999999 0.999999],[0.582549 0.123754 0.586041 0.832796 0.045121],[0.046316 0.393338 0.197938 0.174495 0.336343 ...,[0.067737 0.079147 0.027514 0.046544 0.461641 ...,2.380231e-87,0.047852,-0.005920,-1.917274,4462.544669,-0.579251,1.286160,9.799602
1,45,2024-05-07 07:25:14,534,[0.877228 0.758174],[0.843164 0.999999],[0.509702 0.39142 0.489854],[0.426175 0.38531 0.370372 0.358571],[0.322832 0.999999 0.999999 0.999999],[0.466039 0.148505 0.468833 0.666237 0.036097],[0.046316 0.393338 0.197938 0.174495 0.360367 ...,[0.081284 0.094976 0.033017 0.055853 0.369313 ...,-1.758338e-55,-0.000932,-0.019979,0.563498,4484.415611,-0.763791,1.708912,9.696555
2,82,2024-05-13 06:14:44,534,[0.785666 0.5864 ],[0.769352 0.892841],[0.371119 0.39142 0.489854],[0.34094 0.363106 0.444446 0.430285],[0.387398 0.999999 0.999999 0.999999],[0.466039 0.148505 0.703249 0.999355 0.036097],[0.037053 0.472006 0.15835 0.139596 0.319182 ...,[0.081284 0.094976 0.033017 0.055853 0.553969 ...,-9.346589e-20,0.266689,-0.012444,0.180900,4528.747422,-0.569922,1.033901,9.817107
3,206,2024-06-10 07:09:12,534,[0.800434 0.811483],[0.687022 0.999999],[0.422794 0.313136 0.498535],[0.34094 0.431025 0.296298 0.430285],[0.464878 0.999999 0.999999 0.999999],[0.372831 0.118804 0.562599 0.999999 0.043316],[0.037053 0.31467 0.237526 0.139596 0.43244 ...,[0.097541 0.113971 0.03962 0.067024 0.443175 ...,1.381414e-44,0.589400,-0.027751,-0.728106,4619.357394,-0.625573,1.401136,9.849910
4,265,2024-06-20 05:37:21,534,[0.824063 0.666364],[0.672828 0.952955],[0.556679 0.473172 0.489854],[0.485406 0.308248 0.444446 0.430285],[0.557854 0.999999 0.999999 0.999999],[0.494966 0.118804 0.843899 0.806398 0.043316],[0.037053 0.31467 0.15835 0.209394 0.288294 ...,[0.117049 0.136765 0.047544 0.080429 0.53181 ...,1.055608e-30,0.586961,-0.022115,-1.723890,4806.634380,-0.672301,2.145971,9.859542
5,307,2024-06-25 06:16:12,534,[0.700011 0.811483],[0.740962 0.999999],[0.668015 0.567806 0.587825],[0.390058 0.462372 0.444446 0.352494],[0.669425 0.999999 0.999999 0.999999],[0.634778 0.099003 0.703249 0.999355 0.054145],[0.029642 0.251736 0.12668 0.251273 0.230635 ...,[0.140459 0.109412 0.057053 0.096515 0.425448 ...,-1.535327e-27,0.400386,-0.043506,-0.576021,5200.648778,-0.788500,2.110076,9.833724
6,325,2024-06-28 04:06:03,534,[0.762037 0.77002 ],[0.685634 0.999999],[0.611642 0.358719 0.471252],[0.381391 0.365718 0.366606 0.430285],[0.80331 0.999999 0.999999 0.999999],[0.51425 0.148505 0.614333 0.815566 0.054145],[0.029642 0.251736 0.12668 0.167515 0.288294 ...,[0.093639 0.164118 0.057053 0.096515 0.486226 ...,4.857872e-27,0.633161,-0.028291,0.527600,6055.632249,-0.401099,2.025039,9.865061
7,361,2024-07-01 14:33:18,534,[0.753176 0.78779 ],[0.699513 0.999999],[0.611642 0.37061 0.587825],[0.34094 0.318697 0.444446 0.303874],[0.963972 0.999999 0.999999 0.999999],[0.4114 0.178206 0.635517 0.776194 0.064974],[0.029642 0.350632 0.12668 0.251273 0.313005 ...,[0.074911 0.131294 0.068464 0.115818 0.555687 ...,2.058824e-29,0.557300,-0.044824,-1.277253,8013.336006,-0.292817,2.078230,9.906853
8,425,2024-07-09 17:19:13,534,[0.877228 0.5864 ],[0.822761 0.999999],[0.556679 0.39142 0.32657 ],[0.34094 0.308248 0.296298 0.286857],[0.999999 0.999999 0.999999 0.999999],[0.32912 0.142565 0.76262 0.931433 0.051979],[0.029642 0.377604 0.12668 0.167515 0.230635 ...,[0.059929 0.105035 0.054771 0.092654 0.666824 ...,1.227291e-45,-0.083219,-0.057986,-2.760482,8662.405001,-0.621793,1.371910,9.947503
9,493,2024-07-16 17:14:47,534,[0.877228 0.663402],[0.672663 0.999999],[0.296895 0.445922 0.54566 ],[0.34094 0.397065 0.371627 0.362217],[0.999999 0.999999 0.999999 0.999999],[0.32912 0.213847 0.622368 0.760135 0.077969],[0.029642 0.251736 0.19002 0.251273 0

In [124]:
# LOAD INITIAL DATA PROVIDED AT THE BEGINNING 
def load_data(dim):
    X = np.load('data/initial_data/function_' + str(dim) +'/initial_inputs.npy')
    y = np.load('data/initial_data/function_' + str(dim) +'/initial_outputs.npy')
    return X,y

# LOAD AND APPEND THE RESULTS PUBLISHED BY CARLTON 
X, y = load_data(file)
dimension = np.shape(X)[1] 

for item in input:
     item_with_comma = "[" + item.replace(" 0.",", 0.")  + "]"  
     row = np.array(literal_eval(item_with_comma))
     X = np.append(X, np.array(row), axis=0)

for item in output:
    y = np.append(y, float(item))

print("X(shape)=", X.shape, "y(shape)=", y.shape)


X(shape)= (50, 8) y(shape)= (50,)


In [125]:
X

array([[0.60499445, 0.29221502, 0.90845275, 0.35550624, 0.20166872,
        0.57533801, 0.31031095, 0.73428138],
       [0.17800696, 0.56622265, 0.99486184, 0.21032501, 0.32015266,
        0.70790879, 0.63538449, 0.10713163],
       [0.00907698, 0.81162615, 0.52052036, 0.07568668, 0.26511183,
        0.09165169, 0.59241515, 0.36732026],
       [0.50602816, 0.65373012, 0.36341078, 0.17798105, 0.0937283 ,
        0.19742533, 0.7558269 , 0.29247234],
       [0.35990926, 0.24907568, 0.49599717, 0.70921498, 0.11498719,
        0.28920692, 0.55729515, 0.59388173],
       [0.77881834, 0.0034195 , 0.33798313, 0.51952778, 0.82090699,
        0.53724669, 0.5513471 , 0.66003209],
       [0.90864932, 0.0622497 , 0.23825955, 0.76660355, 0.13233596,
        0.99024381, 0.68806782, 0.74249594],
       [0.58637144, 0.88073573, 0.74502075, 0.54603485, 0.00964888,
        0.74899176, 0.23090707, 0.09791562],
       [0.76113733, 0.85467239, 0.38212433, 0.33735198, 0.68970832,
        0.30985305, 0.631379

In [126]:
y


array([7.3987211 , 7.00522736, 8.45948162, 8.28400781, 8.60611679,
       8.54174792, 7.32743458, 7.29987205, 7.95787474, 5.59219339,
       7.85454099, 6.79198578, 8.97655402, 7.3790829 , 9.598482  ,
       8.15998319, 7.13162397, 6.76796253, 7.43374407, 9.01307515,
       7.31089382, 5.84106731, 9.14163949, 8.81755844, 6.45194313,
       8.83074505, 9.34427428, 6.88784639, 8.04221254, 7.69236805,
       7.92375877, 8.42175924, 8.2780624 , 7.11345716, 6.40258841,
       8.47293632, 7.97768459, 7.46087219, 7.43659353, 9.18300525,
       9.79960224, 9.69655457, 9.81710696, 9.84991022, 9.85954215,
       9.83372387, 9.86506113, 9.90685296, 9.94750315, 9.95332045])

In [127]:
# The first 2 functions can be shown as X,Y coordinates and reveal what is going on. On higher dimensions it loses meaning so we don't plot it 
if (dimension == 1 or dimension == 2):
    x1 =X[:,0]
    x2 =X[:,1] 
    plt.scatter(x1, x2, c=y)
    for (i, j, text) in zip(x1, x2, y):
        plt.annotate(format(text, ".2e"),  # This is the text to use for the annotation
                 (i, j),  # This is the point to label
                 textcoords="offset points",  # how to position the text
                 xytext=(0,10),  # distance from text to points (x,y)
                 ha='right')  # horizontal alignment can be left, right or center
    plt.show()



# Upper Confidence Bound

A second alternative would be to use Bayesian Optimization and consider an Upper Confidence Bound acquisition function: 

In [128]:
# CHANGED FROM MANUAL TO BAYES OPTIMIZATION 

from skopt import BayesSearchCV
from skopt.space import Real, Categorical


estimator_list = [GaussianProcessRegressor(kernel=C(1.0, constant_value_bounds="fixed") * RBF(1.0, length_scale_bounds="fixed")),
                  GaussianProcessRegressor(kernel=C(1.0, constant_value_bounds="fixed") * Matern(length_scale=1.0, nu=1.5)),
                  GaussianProcessRegressor(kernel=C(1.0, constant_value_bounds="fixed") * RationalQuadratic(length_scale=1.0, alpha=1.5)),
                  GaussianProcessRegressor(kernel=C(1.0, constant_value_bounds="fixed") * ExpSineSquared(length_scale=1, periodicity=1)),
                  GaussianProcessRegressor(kernel=C(1.0, constant_value_bounds="fixed") * DotProduct(sigma_0=1.0))]

pipe=Pipeline([('estimator',GaussianProcessRegressor())])

param = {
    'estimator': Categorical(estimator_list),
    'estimator__n_restarts_optimizer': (5,10),
    'estimator__alpha': (1e-5, 1e-2,'log-uniform')
}

bayes_search = BayesSearchCV(
    estimator=pipe,
    search_spaces=param,
    cv=3,
    scoring="r2",
    random_state=42,
    n_iter=25,
    verbose=1,
)  

bayes_search.fit(X, y)

#kernels = [
#    C(1.0, constant_value_bounds="fixed") * RBF(1.0, length_scale_bounds="fixed"),
#    C(1.0, (1e-2, 1e2)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2), nu=1.5),
#    C(1.0, (1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, alpha=0.1, alpha_bounds=(1e-2, 1e2)),
#    C(1.0, (1e-2, 1e2)) * ExpSineSquared(length_scale=1.0, periodicity=3.0, periodicity_bounds=(1e-2, 1e1)),
#    C(1.0, constant_value_bounds="fixed") * DotProduct(sigma_0=1.0)
#]
#
## Define the search space for a single kernel type (RBF in this example)
#search_space = {
#    'kernel__k1__constant_value': Real(1e-2, 1e2, prior='log-uniform'),
#    'kernel__k2__length_scale': Real(1e-2, 1e2, prior='log-uniform')
#}
#
## Create the Gaussian Process Regressor with an initial kernel (RBF in this example)
#gpr = GaussianProcessRegressor(kernel=kernels[0])
#
## Create a Bayesian optimization search object
#bayes_search = BayesSearchCV(estimator=gpr, search_spaces=search_space, n_iter=32, cv=5, scoring='neg_mean_squared_error', random_state=0)
#
## Fit the Bayesian search to the data
#bayes_search.fit(X, y)

# Print the best parameters and best score
print("Best parameters found: ", bayes_search.best_params_)
print("Best score: ", bayes_search.best_score_)

# Update the GPR with the best kernel
gpr = bayes_search.best_estimator_


print(gpr)



Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\vhar1\anaconda3\Lib\site-packages\skle

Fitting 3 folds for each of 1 candidates, totalling 3 fits


c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

Fitting 3 folds for each of 1 candidates, totalling 3 fits


c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\vhar1\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters found:  OrderedDict([('estimator', GaussianProcessRegressor(alpha=0.0021188154764215273,
                         kernel=1**2 * ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=7)), ('estimator__alpha', 0.0021188154764215273), ('estimator__n_restarts_optimizer', 7)])
Best score:  0.95877540

In [129]:
def find_maxiumum(X, y):
    maximum_of_X = []
    maximum_so_far = -1e99
    for index in range(len(y)):
        if (y[index] > maximum_so_far):
            maximum_so_far = y[index]
            maximum_of_X = []
            for i in range(dimension):
                maximum_of_X.append(X[index, i])
    return np.array(maximum_of_X), maximum_so_far
    
discovery_radius = 0.2   # on each datapoint use 20% discovery radius

Xmax, ymax = find_maxiumum(X, y)  #Returns the row with maximim y and all X values 
XmaxLower = Xmax - (Xmax * discovery_radius)
XmaxUpper = Xmax + (Xmax * discovery_radius)
print("maximum X=", Xmax, ", maximum y=", ymax)

# Adjust sizes as we go up in dimensions to avoid waiting for hours on higher dimensions 
# ********* For the purpose of unblocking any memory issue please decrease these values to adjust to your computer specs ******************
sizes = [ [2, 100],
          [3,  80],
          [4,  60],
          [5,  30],
          [6,  15],
          [7,  10],
          [8,  8]]

size = sizes[dimension - 2][1]  #Find the corrects size for this dimension 

data = []
for item in range(dimension):
    #  Upper lower min and max explanation: 
    #  This iteration (each X value) creates a number of points of size "size" in a radius of +/-20% and ensures this radius is between 0 and 1, e.g.  
    # - X values [0.5, 0, 0.99] 
    # - apply 20% discovery radius => [(0.4 to 0.6), (0 to 0), (0.792 to 1.188)] 
    # - apply 0 to 1 filter => [ linspace(0.4 to 0.6), linspace(0 to 0.0001), linspace(9.792 to 9.9999)]
    # - result = [ [0.48777254 ... 0.59072617], [0.000002, ... 0.000099], [9.7921111, ...9.998888] ]
    if (XmaxUpper[item] == 0):
        XmaxUpper[item] = 0.0001
    if (XmaxLower[item] == 1):
        XmaxLower[item] = 0.9999
    data.append(np.linspace(max(XmaxLower[item], 0 + 1e-10), min(XmaxUpper[item], 1 - 1e-10), size))

x_grid_points = np.array(data)

print("data-points=", x_grid_points.shape)
for dim in range(dimension):
    print("data-points[", dim + 1, "]=", x_grid_points[dim][:5], "...", x_grid_points[dim][-1] )



maximum X= [0.071915 0.084028 0.065725 0.074123 0.647772 0.463092 0.172738 0.885714] , maximum y= 9.9533204500384
data-points= (8, 8)
data-points[ 1 ]= [0.057532   0.06164143 0.06575086 0.06986029 0.07396971] ... 0.08629800000000001
data-points[ 2 ]= [0.0672224 0.072024  0.0768256 0.0816272 0.0864288] ... 0.10083360000000001
data-points[ 3 ]= [0.05258    0.05633571 0.06009143 0.06384714 0.06760286] ... 0.07887000000000001
data-points[ 4 ]= [0.0592984 0.063534  0.0677696 0.0720052 0.0762408] ... 0.08894759999999999
data-points[ 5 ]= [0.5182176  0.55523314 0.59224869 0.62926423 0.66627977] ... 0.7773264
data-points[ 6 ]= [0.3704736 0.396936  0.4233984 0.4498608 0.4763232] ... 0.5557104
data-points[ 7 ]= [0.1381904  0.14806114 0.15793189 0.16780263 0.17767337] ... 0.20728560000000001
data-points[ 8 ]= [0.7085712  0.75020389 0.79183657 0.83346926 0.87510194] ... 0.9999999999


In [130]:
# Apply grid search = cartesian product of the array, e.g. data of shape (2,100) => (100 x 100) = (10000, 2)
X_grid = np.fromiter(it.chain(*it.product(*x_grid_points)), dtype=float).reshape(-1,dimension)
print("X_grid=", X_grid.shape, " = ", X_grid[0:5,:], "...", X_grid[-1,:])


X_grid= (16777216, 8)  =  [[0.057532   0.0672224  0.05258    0.0592984  0.5182176  0.3704736
  0.1381904  0.7085712 ]
 [0.057532   0.0672224  0.05258    0.0592984  0.5182176  0.3704736
  0.1381904  0.75020389]
 [0.057532   0.0672224  0.05258    0.0592984  0.5182176  0.3704736
  0.1381904  0.79183657]
 [0.057532   0.0672224  0.05258    0.0592984  0.5182176  0.3704736
  0.1381904  0.83346926]
 [0.057532   0.0672224  0.05258    0.0592984  0.5182176  0.3704736
  0.1381904  0.87510194]] ... [0.086298  0.1008336 0.07887   0.0889476 0.7773264 0.5557104 0.2072856
 1.       ]


In [131]:
mean, std = gpr.predict(X_grid, return_std = True)

def calculate_ucb(beta):   
    ucb = mean + beta * std
    print("UCB=", ucb.shape, ", value=", ucb[:5])
    print("mean=", mean.shape, ", value=", mean[:5])
    idx_max = np.argmax(ucb)   # take the index of the highest confidence result 
    next_query = X_grid[idx_max]    # next_query is the point with highest confidence in this grid of search 
    best_numbers = []
    for item in next_query:
        best_numbers.append(min(max(item, 0.000001), 0.999999))   # ensure the numbers are between 0 and 1 
    print(best_numbers)
    return best_numbers

def print_results(label, best_numbers):
    #Formats the values in a way it can be copied and pasted into the weekly submission form 
    print(label)
    print(np.array2string(np.array(best_numbers), precision=6, separator='-', floatmode='fixed', formatter={'float': '{:0.6f}'.format}))

print("balanced")
balanced = calculate_ucb(1.96)  #keep it balanced

print("\n\nexploitative")
exploitative = calculate_ucb(1.0)   #lets exploit target area

print("\n\nexploratory")
exploratory = calculate_ucb(2.5)   #lets explore more 



balanced
UCB= (16777216,) , value= [10.0013482   9.99328852  9.98320213  9.97126749  9.95762638]
mean= (16777216,) , value= [9.88518573 9.88166502 9.87440602 9.86338876 9.84859469]
[0.08629800000000001, 0.10083360000000001, 0.07887000000000001, 0.08894759999999999, 0.7032953142857143, 0.44986080000000006, 0.20728560000000001, 0.7085712]


exploitative
UCB= (16777216,) , value= [9.9444523  9.93861578 9.92991424 9.91842893 9.90422311]
mean= (16777216,) , value= [9.88518573 9.88166502 9.87440602 9.86338876 9.84859469]
[0.08629800000000001, 0.10083360000000001, 0.07887000000000001, 0.08894759999999999, 0.6662797714285714, 0.44986080000000006, 0.20728560000000001, 0.7085712]


exploratory
UCB= (16777216,) , value= [10.03335214 10.02404193 10.01317657 10.00098918  9.98766573]
mean= (16777216,) , value= [9.88518573 9.88166502 9.87440602 9.86338876 9.84859469]
[0.08629800000000001, 0.10083360000000001, 0.07887000000000001, 0.08894759999999999, 0.7773264, 0.44986080000000006, 0.2072856000000000

In [132]:
print(bayes_search.best_params_)

print_results("exploitative", exploitative)

print_results("\nbalanced", balanced)

print_results("\nexploratory", exploratory)


OrderedDict([('estimator', GaussianProcessRegressor(alpha=0.0021188154764215273,
                         kernel=1**2 * ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=7)), ('estimator__alpha', 0.0021188154764215273), ('estimator__n_restarts_optimizer', 7)])
exploitative
[0.086298-0.100834-0.078870-0.088948-0.666280-0.449861-0.207286-0.708571]

balanced
[0.086298-0.100834-0.078870-0.088948-0.703295-0.449861-0.207286-0.708571]

exploratory
[0.086298-0.100834-0.078870-0.088948-0.777326-0.449861-0.207286-0.708571]
